In [1]:
using LinearAlgebra
using SparseArrays
using Statistics
import Laplacians as Lap
include("Laplacians.jl/src/Laplacians.jl")
import .Laplacians as LapM
import Random
using Plots
using BenchmarkTools
using CUDA
using Graphs
using SNAPDatasets
using GraphIO
using PProf

In [2]:
function loadMatCSCfromBinary(fname)
    mn = Array{Int64, 1}(undef, 2)
    open(fname, "r") do file
      read!(file, mn)
    end
    m = mn[1]
    n = mn[2]
    colptr = Array{Int64, 1}(undef, n+1)
    open(fname, "r") do file
      read!(file, mn)
      read!(file, colptr)
    end
    rowval = Array{Int64, 1}(undef, colptr[n+1]-1)
    nzval = Array{Float64, 1}(undef, colptr[n+1]-1)
    open(fname, "r") do file
      read!(file, mn)
      read!(file, colptr)
      read!(file, rowval)
      read!(file, nzval)
    end
    spmat = SparseMatrixCSC{Float64, Int64}(m, n, colptr, rowval, nzval)
    return spmat
end

function loadRandNumsFromBinary(fname)
    rnums = Array{Float64, 1}(undef, 5000000)
    open(fname, "r") do file
        read!(file, rnums)
    end
    return rnums
end

function loadLDLiout(fname)
    szs = Array{Int64, 1}(undef, 5)
    open(fname, "r") do file
        read!(file, szs)
    end
    csz, cptrsz, rvsz, fvsz, dsz = szs
    col = Vector{Int64}(undef, csz)
    colptr = Vector{Int64}(undef, cptrsz)
    rowval = Vector{Int64}(undef, rvsz)
    fval = Vector{Float64}(undef, fvsz)
    d = Vector{Float64}(undef, dsz)
    
    open(fname, "r") do file
        read!(file, szs)
        read!(file, col)
        read!(file, colptr)
        read!(file, rowval)
        read!(file, fval)
        read!(file, d)
    end    
    return LapM.LDLinv{Int64, Float64}(col .+ 1, colptr .+ 1, rowval .+ 1, fval, d)
end

function cmpldli(oldli, ldli)
    println("maximum abs difference for col: $(maximum(abs.(oldli.col - ldli.col .+ 1)))")
    println("maximum abs difference for colptr: $(maximum(abs.(oldli.colptr - ldli.colptr .+ 1)))")
    println("maximum abs difference for rowval: $(maximum(abs.(oldli.rowval - ldli.rowval .+ 1)))")
    println("maximum abs difference for fval: $(maximum(abs.(oldli.fval - ldli.fval)))")
    println("maximum abs difference for d: $(maximum(abs.(oldli.d - ldli.d)))")
end

cmpldli (generic function with 1 method)

In [19]:
oldli = loadLDLiout("Laplacians.jl/ldliout.bin")
# println("oldli.col: $(oldli.col), len: $(length(oldli.col))")
# println("oldli.colptr: $(oldli.colptr), len: $(length(oldli.colptr))")
# println("oldli.d: $(oldli.d), len: $(length(oldli.d))")
# println("oldli.fval: $(oldli.fval), len: $(length(oldli.fval))")
# println("oldli.rowval: $(oldli.rowval), len: $(length(oldli.rowval))")

Main.Laplacians.LDLinv{Int64, Float64}([44635, 1568, 1777, 2479, 2993, 3022, 3309, 3316, 3570, 3660  …  148, 20, 23, 40, 3, 13, 14, 39, 11, 399], [1, 2, 4, 6, 8, 10, 12, 14, 16, 18  …  441140, 441149, 441157, 441164, 441170, 441175, 441179, 441182, 441184, 441185], [17278, 358, 863, 1152, 1253, 585, 2087, 442, 2221, 1156  …  39, 16, 11, 399, 399, 16, 11, 16, 399, 16], [1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5  …  0.06132433783353129, 0.32174541299508547, 0.4802133188631721, 1.0, 0.1765832006231307, 0.25869174636701714, 1.0, 0.41711670484345037, 1.0, 1.0], [0.25236782477325365, 0.3189264055372324, 2.059243275034341, 0.3778493521842823, 0.09409202131134392, 0.16712003340476006, 0.16462485761344958, 0.42577258668767376, 0.33156464999137886, 0.10526045151013487  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [20]:
rnums = loadRandNumsFromBinary("Laplacians.jl/randomNums.bin")

5000000-element Vector{Float64}:
 0.5361670556581197
 0.8160044081547981
 0.2005106696682415
 0.6229061332345399
 0.4423113630390252
 0.9717385222163453
 0.2628360482165598
 0.5464355193894663
 0.35019247435776824
 0.9509418540357933
 0.9118146057006138
 0.15097115760293678
 0.6908575391858608
 ⋮
 0.9787807351651087
 0.38102549497801463
 0.5355221246916624
 0.06796548065692487
 0.266162816257837
 0.16487560574631122
 0.1545447928321524
 0.03038541144712159
 0.7762531901448376
 0.8285272236868789
 0.5912004422712964
 0.1723968281605035

In [21]:
G = loadMatCSCfromBinary("Laplacians.jl/small.bin")

50000×50000 SparseMatrixCSC{Float64, Int64} with 199972 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

In [22]:
LapMLLmat = LapM.LLmatp(G)
reorder = collect(range(1, length(LapMLLmat.degs)))
perm = sortperm(LapMLLmat.degs)
permute!(reorder, perm)
ldli = LapM.approxChol_reordered(LapMLLmat, reorder, rnums)
# ldli, log = LapM.approxChol_log(LapMLLmat)
# println("reorder: $(reorder)")
# println("--------------------")
# println("ldli.col: $(ldli.col), len: $(length(ldli.col))")
# println("ldli.colptr: $(ldli.colptr), len: $(length(ldli.colptr))")
# println("ldli.d: $(ldli.d), len: $(length(ldli.d))")
# println("ldli.fval: $(ldli.fval), len: $(length(ldli.fval))")
# println("ldli.rowval: $(ldli.rowval), len: $(length(ldli.rowval))")


Main.Laplacians.LDLinv{Int64, Float64}([44635, 1568, 1777, 2479, 2993, 3022, 3309, 3316, 3570, 3660  …  148, 20, 23, 40, 3, 13, 14, 39, 11, 399], [1, 2, 4, 6, 8, 10, 12, 14, 16, 18  …  440652, 440661, 440669, 440676, 440682, 440687, 440691, 440694, 440696, 440697], [17278, 358, 863, 1152, 1253, 585, 2087, 442, 2221, 1156  …  39, 11, 16, 399, 16, 399, 11, 16, 399, 16], [1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5  …  0.13129250869022327, 0.2178492531642519, 0.44439039899883415, 1.0, 0.2547294840056442, 0.44433089647358825, 1.0, 0.37809593105135036, 1.0, 1.0], [0.2721922668606747, 0.29414512899300055, 1.0020499630191817, 0.4091398152945027, 0.13375274413233218, 0.20827173927629886, 0.15462570284344151, 0.45981527773663916, 0.33729146669899546, 0.09670914126227668  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [8]:
applyLDLi(ldli, mat) = sparse(reduce(hcat, map(x -> LapM.LDLsolver(ldli, Vector(x)), eachcol(mat))))

applyLDLi (generic function with 1 method)

In [23]:
LapM.preprocessLDL(ldli)
LapM.preprocessLDL(oldli)

49999-element Vector{Int64}:
  1
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  2
  ⋮
 12
 11
 10
  9
  8
  7
  6
  5
  4
  3
  2
  1

In [24]:
println(LapM.condNumber(G, oldli)) 
println(LapM.condNumber(G, ldli))

4.069700871960519
4.452845997950618


In [14]:
LapM.LDLsolver(oldli, Vector(LapM.lap(G)[:, 3]))

50000-element Vector{Float64}:
  0.005252831725038529
 -0.010516587108028691
  1.1102422534895025
  0.03092598621229488
  0.0014572077846086162
  0.010615488914343094
  0.029298591879472632
 -0.02247541200493368
  0.040858022126301116
  0.005893724994370728
  0.034450315572929754
  0.005537650917774861
  0.14284020123944635
  ⋮
  0.0002287037052585289
  0.007124124274444935
  0.000629900992170182
 -0.0005845470696366623
 -0.0006668439054706016
  0.0006128067587651137
  0.001225205803752502
 -0.0010106219991242865
  0.005575359836329438
 -0.0004074057419242716
 -0.0016347882460999302
  0.0003089885946094967